In [1]:
# install packages

import pandas as pd

!pip install lxml

!pip install geocoder

import geocoder

import csv

In [2]:
# Scrape Wikipedia page for Toronto postal code data

postal_codes = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',skiprows=1)[0]

In [3]:
# Transofrm web page data into dataqframe

postal_codes.columns = ["Postal Code", "Borough", "Neighborhood"]
postal_codes.head()

,Postal Code,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights


In [4]:
# Remove 'Not assigned'

postal_codes=postal_codes[postal_codes.Borough != 'Not assigned']

postal_codes.head()

,Postal Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor


In [5]:
# Combine rows with same postal code

df = postal_codes.groupby(by=['Postal Code','Borough']).agg(lambda x: ', '.join(x)) 
df.reset_index(level=['Postal Code','Borough'], inplace=True) 

df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# print number of rosw. Now we have 103 rows. 

df.shape

(103, 3)

In [8]:
# initialize variable to None
lat_lng_coords = None

# loop to get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(df))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [ ]:
# convert into existing dataframe
df['Latitude']=0
df['Longitude']=0
for i, url in enumerate(url2):
    r = requests.get(url2[i], params=params)
    results = r.json()['results']
    if results:
        location = results[0]['geometry']['location']
    else:
        print ("no results")
    df['Latitude'].iloc[i] = location['lat']
    df['Longitude'].iloc[i]= location['lng']df.head(10)
    
df.head()

In [7]:
# merge coordinates into dataset with csv file

geo_df = pd.read_csv('http://cocl.us/Geospatial_data')

df = pd.merge(df, geo_df, how="right", on="Postal Code")

df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
